# Постановка экспериментов на воспроизводимость модели

Загрузка нужных библиотек для работы

In [5]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.4 MB/s eta 0:00:00


In [6]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.8 MB/s eta 0:00:00


In [13]:
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings("ignore")
from catboost import CatBoostRegressor
import optuna
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from sklearn.pipeline import Pipeline

In [8]:
# Загрузка сохраненной модели из файла pickle
with open("/content/drive/MyDrive/cbr_model_optuna.pkl", "rb") as f:
    model_for_work = pickle.load(f)

In [10]:
# создадим набор данных для прогнозирования
data = [
    ('Pending',	2.0,True,	'Brook Park',	936.0,'44142','OH',		'single_family',	False,	'1958',	True,	True,	False,	4.4,	0.70),
    ('Active',	2.0,True,	'Fresno',	1650.0,	'93706','CA',		'single_family',	False,	'1931',	False,	False,	False, 4.0,	0.87),
    ('Active', 5.0, False,'Dallas', 2300.0, '75205','TX',  'single_family', False, '2006', True, True, True, 6.7, 1.19),
    ('Active', 3.0,False,'Kansas',	2058.0,	'64112','MO',		'condos',	False,	'2008',	True,	True,	False,	2.3,	2.70),
    ('Active', 4.0, False, 'Palm Bay', 2990.0, '75205','FL',  'single_family', False, '2006', True, True, True, 6.7, 1.19)
]

columns = ['status','baths','fireplace','city','sqft','zipcode','state','Type','pool','Year built','Heating','Cooling','Parking','school_rating _mean', 'school_dist_min']

df_pred = pd.DataFrame(data, columns=columns)

df_pred.head()

,status,baths,fireplace,city,sqft,zipcode,state,Type,pool,Year built,Heating,Cooling,Parking,school_rating _mean,school_dist_min
0,Pending,2.0,True,Brook Park,936.0,44142,OH,single_family,False,1958,True,True,False,4.4,0.70
1,Active,2.0,True,Fresno,1650.0,93706,CA,single_family,False,1931,False,False,False,4.0,0.87
2,Active,5.0,False,Dallas,2300.0,75205,TX,single_family,False,2006,True,True,True,6.7,1.19
3,Active,3.0,False,Kansas,2058.0,64112,MO,condos,False,2008,True,True,False,2.3,2.70
4,Active,4.0,False,Palm Bay,2990.0,75205,FL,single_family,False,2006,True,True,True,6.7,1.19


In [23]:
#Организация кастомного трансформера,который наследуется от двух классов: TransformerMixin и BaseEstimator.
class LogDataTransformer(TransformerMixin, BaseEstimator):

    def __init__(self):    #инициализация параметров, не зависящих от данных
      pass

    def fit(self, X, y=None):  # обучение трансформера

      return self

    def transform(self, X):   # трансформация приходящих на вход

      X_transformed = X.copy()
      X_transformed['zipcode'] = X_transformed['zipcode'].astype(str)
      X_transformed['Year built'] = X_transformed['Year built'].astype(str)

      for column in ['baths', 'sqft', 'school_rating _mean', 'school_dist_min']:
          constant = 1e-6
          X_transformed[column] = X_transformed[column].apply(lambda x: abs(x))   #нормализация и логорифмирование численных значений
          X_transformed[column] = np.log(X_transformed[column] + constant)

      return X_transformed

In [24]:
#пайплайн,где во первых выполняется  трансформация, а затем модель обучается на полученных данных

preprocessing_pipeline = Pipeline([
    ('log_data', LogDataTransformer()),
    ('model', model_for_work)
])

In [30]:
#предсказания значений

predictions = preprocessing_pipeline.predict(df_pred)
rounded_predictions = np.round(np.exp(predictions))
print(rounded_predictions)

[110423. 292852. 710681. 505165. 736758.]


**Мы видим,что модель воспроизводится на новых данных.
И даже без углубленного анализа видно, что чем больше площадь,чем выше цена на объект недвижимости.**